<a href="https://colab.research.google.com/github/Malik-Raheel/Trading-Strategies/blob/main/RSI_Below_30_Signals_for_Alpha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Assets Tracking for Entries for Day Trading

In [1]:
pip install ccxt pandas numpy ta scikit-learn matplotlib seaborn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 935.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.6/288.6 kB 11.6 MB/s eta 0:00:00
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=9161c5e079e562161a48a127324bfe6d82b5c9e05351f8755c49922cf3c2914c
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.11
    Uninstalling aiohttp-3.11.11:
      Successfully uninstalled aiohttp-3.11.11


In [4]:
import ccxt
import pandas as pd
from ta.momentum import RSIIndicator
from ta.trend import EMAIndicator

# Initialize exchange
exchange = ccxt.kucoin()  # Replace with your preferred exchange

# List of assets
symbols = [
    "BTC/USDT", "ETH/USDT", "AVAX/USDT", "SOL/USDT", "XAVA/USDT",
    "SLIM/USDT", "HNT/USDT", "NEAR/USDT", "DOGE/USDT", "SAND/USDT",
    "CRV/USDT", "CFX/USDT", "INJ/USDT", "LINK/USDT", "AAVE/USDT",
    "LTC/USDT", "ADA/USDT", "GAFI/USDT", "JUP/USDT", "MNT/USDT",
    "PYTH/USDT", "STG/USDT"
]

# Function to fetch historical data
def fetch_ccxt_data(symbol, timeframe, limit=500):
    """Fetch historical OHLCV data using CCXT."""
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=["time", "open", "high", "low", "close", "volume"])
    df["time"] = pd.to_datetime(df["time"], unit="ms")
    return df

# Function to calculate RSI and EMA
def calculate_indicators(df, rsi_period=14, ema_period=200):
    """Add RSI and EMA indicators to the DataFrame."""
    df["RSI"] = RSIIndicator(df["close"], window=rsi_period).rsi()
    ema = EMAIndicator(df["close"], window=ema_period)
    df["EMA"] = ema.ema_indicator()
    return df

# Signal tracking function
def track_signals():
    """Track assets that meet the criteria."""
    signals = []

    for symbol in symbols:
        try:
            print(f"Processing {symbol}...")

            # Fetch data for required timeframes
            data_15m = fetch_ccxt_data(symbol, "15m")
            data_1h = fetch_ccxt_data(symbol, "1h")
            data_4h = fetch_ccxt_data(symbol, "4h")
            data_1d = fetch_ccxt_data(symbol, "1d")
            data_1w = fetch_ccxt_data(symbol, "1w")

            # Calculate indicators
            data_15m = calculate_indicators(data_15m)
            data_1h = calculate_indicators(data_1h)
            data_4h = calculate_indicators(data_4h)
            data_1d = calculate_indicators(data_1d)
            data_1w = calculate_indicators(data_1w)

            # Check RSI condition
            rsi_15m = data_15m["RSI"].iloc[-1] < 30
            rsi_1h = data_1h["RSI"].iloc[-1] < 30
            rsi_4h = data_4h["RSI"].iloc[-1] < 30
            rsi_1d = data_1d["RSI"].iloc[-1] < 30

            # Check EMA condition
            price_above_ema_1d = data_1d["close"].iloc[-1] > data_1d["EMA"].iloc[-1]
            price_above_ema_1w = data_1w["close"].iloc[-1] > data_1w["EMA"].iloc[-1]

            # Signal criteria
            if (
                (rsi_15m or rsi_1h or rsi_4h or rsi_1d)
                and (price_above_ema_1d or price_above_ema_1w)
            ):
                signals.append({
                    "symbol": symbol,
                    "RSI_15m": data_15m["RSI"].iloc[-1],
                    "RSI_1h": data_1h["RSI"].iloc[-1],
                    "RSI_4h": data_4h["RSI"].iloc[-1],
                    "RSI_1d": data_1d["RSI"].iloc[-1],
                    "Price": data_1d["close"].iloc[-1],
                    "EMA_1d": data_1d["EMA"].iloc[-1],
                    "EMA_1w": data_1w["EMA"].iloc[-1],
                })

        except Exception as e:
            print(f"Error processing {symbol}: {e}")

    return signals

# Main execution
if __name__ == "__main__":
    signals = track_signals()
    if signals:
        print("\nSignals Found:")
        for signal in signals:
            print(signal)
    else:
        print("\nNo signals found based on the given criteria.")


Processing BTC/USDT...
Processing ETH/USDT...
Processing AVAX/USDT...
Processing SOL/USDT...
Processing XAVA/USDT...
Processing SLIM/USDT...
Processing HNT/USDT...
Processing NEAR/USDT...
Processing DOGE/USDT...
Processing SAND/USDT...
Processing CRV/USDT...
Processing CFX/USDT...
Processing INJ/USDT...
Processing LINK/USDT...
Processing AAVE/USDT...
Processing LTC/USDT...
Processing ADA/USDT...
Processing GAFI/USDT...
Processing JUP/USDT...
Processing MNT/USDT...
Processing PYTH/USDT...
Processing STG/USDT...

No signals found based on the given criteria.
